Here, the equations to be solved are given as:

$\begin{equation}\begin{array}{c}
A(n)^{2} \partial_{t} \phi&=&-f^{\prime}(\phi)+\lambda B(n) g^{\prime}(\phi) u 
+\frac{1}{2} \nabla_{\Gamma} \cdot \left(|\nabla \phi|^{2} \frac{\partial\left[A(n)^{2}\right]}{\partial \nabla \phi}  
 +A(n)^{2} \nabla_{\Gamma} \phi \right) \\
\partial_{t} u&=&\tilde{D} \nabla_{\Gamma} \cdot\left(q(\phi) \nabla_{\Gamma}\right) u-\frac{L_{\mathrm{sat}}}{2} B(n) \partial_{t} \phi
\end{array}\end{equation}$

Procedure as follows, within a time loop, 

(a) compute $\nabla \phi$

(b) compute $\nabla_{\Gamma} \phi$

(c) compute $M = \left(|\nabla \phi|^{2} \frac{\partial\left[A(n)^{2}\right]}{\partial \nabla \phi}  
 +A(n)^{2} \nabla_{\Gamma} \phi \right)$

(d) compute $GDM = \frac{1}{2} \nabla_{\Gamma} \cdot M$

(e) compute $f'(\phi),~B(n),~A^2(n),~g'(\phi),~T = \frac{\partial\left[A(n)^{2}\right]}{\partial \nabla \phi}$ at each point

(f) compute $\partial_t \phi = (-f'(\phi) + \lambda B(n) g'  u + \frac{1}{2} \nabla_{\Gamma} \cdot M)*(1/A^2(n))$

(g) update $\phi^{n+1} = \phi^n + \Delta t~\partial_t \phi$

(h) compute $q(\phi)$ using new $\phi$

(i) compute $R = \left(q(\phi) \nabla_{\Gamma}\right) u$

(j) compute $\nabla_{\Gamma} \cdot R$

(k) compute $\partial_t u = \tilde{D} \nabla_{\Gamma} \cdot R - \frac{L_{sat}}{2} B(n) \partial_t \phi$

(g) update $u^{n+1} = u^n + \Delta t~\partial_t u$

In [ ]:
from pyevtk.hl import gridToVTK
from datetime import datetime
import numpy
import math
import numba
import warnings
import time
import random

In [ ]:
numeric_tol = 1E-7;

In [ ]:
@numba.njit(fastmath=True)
def compute_A_square(m_del_phi, m_ep_xy, m_ep_z):
    m_n = compute_n(m_del_phi);
    m_theta = math.atan(math.fabs(m_n[0])/(math.fabs(m_n[1])+numeric_tol));
    m_psi = math.atan(math.sqrt(m_n[0]**2+m_n[1]**2)/(math.fabs(m_n[2])+numeric_tol));
    A = 1.+m_ep_xy*math.cos(6.*m_theta)+m_ep_z*math.cos(2.*m_psi);
    return (A**5);

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def compute_n(m_del_phi):
    m_n = [0.,0.,0.];
    m_del_phi_norm = math.sqrt(m_del_phi[0]**2+m_del_phi[1]**2+m_del_phi[2]**2);
    
    for k in range(3):
        m_n[k]= -m_del_phi[k]/(m_del_phi_norm+numeric_tol);
    return m_n;

In [ ]:
@numba.njit(fastmath=True)
def compute_B(m_del_phi, m_gamma):
    m_n = compute_n(m_del_phi);
    return 1.*math.sqrt(m_n[0]**2+m_n[1]**2+m_gamma*m_n[2]**2);

In [ ]:
@numba.njit(fastmath=True)
def compute_f_prime(m_phi):
    return 1.*(m_phi**3-m_phi);

In [ ]:
@numba.njit(fastmath=True)
def compute_g_prime(m_phi):
    return 1.*(1.-m_phi**2)**2;

In [ ]:
@numba.njit(fastmath=True)
def compute_q(m_phi):
    return 1.*(1.-m_phi);

In [ ]:
@numba.njit(fastmath=True)
def compute_G(m_del_phi, m_ep_xy, m_ep_z):
    g = [0.,0.,0.];
    mod_del_phi_local = m_del_phi[0]**2 + m_del_phi[1]**2 + m_del_phi[2]**2;
    ref_z_local = m_del_phi[0]**2 + m_del_phi[1]**2 - m_del_phi[2]**2;
    modxy_del_phi_local = m_del_phi[0]**2 + m_del_phi[1]**2;

    if (mod_del_phi_local>numeric_tol and modxy_del_phi_local>numeric_tol and math.fabs(m_del_phi[0])>numeric_tol  and math.fabs(m_del_phi[1])>numeric_tol  and math.fabs(m_del_phi[2])>numeric_tol):
        g[0] = 4.*(m_ep_z*(ref_z_local) - (m_ep_xy*math.cos(6.*math.atan(m_del_phi[0]/m_del_phi[1])) + 1.)*(mod_del_phi_local))*(-m_ep_z*m_del_phi[0]*(modxy_del_phi_local)*(ref_z_local) + m_ep_z*m_del_phi[0]*(modxy_del_phi_local)*(mod_del_phi_local) + 3.*m_ep_xy*m_del_phi[1]*(mod_del_phi_local)**2*math.sin(6.*math.atan(m_del_phi[0]/m_del_phi[1])))/((modxy_del_phi_local)*(mod_del_phi_local)**3);

        g[1] =  -4.*(m_ep_z*(ref_z_local) - (m_ep_xy*math.cos(6.*math.atan(m_del_phi[0]/m_del_phi[1])) + 1.)*(mod_del_phi_local))*(m_ep_z*m_del_phi[1]*(modxy_del_phi_local)*(ref_z_local) - m_ep_z*m_del_phi[1]*(modxy_del_phi_local)*(mod_del_phi_local) + 3.*m_ep_xy*m_del_phi[0]*(mod_del_phi_local)**2*math.sin(6.*math.atan(m_del_phi[0]/m_del_phi[1])))/((modxy_del_phi_local)*(mod_del_phi_local)**3);

        g[2] = 8.*m_ep_z*m_del_phi[2]*(modxy_del_phi_local)*(-m_ep_z*(ref_z_local) + (m_ep_xy*math.cos(6.*math.atan(m_del_phi[0]/m_del_phi[1])) + 1.)*(mod_del_phi_local))/(mod_del_phi_local)**3;
    return g;

In [ ]:
# constants for simulation
delta = .5;
N_x = 250; N_y = 250; N_z = 1;
delta_t = 5E-3;
n_tstep = 4800;
tmax = n_tstep*delta_t;
sample_rate = 10;
t = 0;

# physical constants
lamd= 3.0;
D_tilde = 0.6267*lamd;
L_sat = random.uniform(1.25,1.35); # range 1, 1.6
ep_xy = 0.2; 
ep_z = random.uniform(0.35,0.45); # range 0.01 to 0.5
Gamma = random.uniform(0.25,0.35); # range 0.1 to 0.5
u_0 = random.uniform(0.7,0.8); # range 0 to 1

In [ ]:
# define a Laplacian calculator 
@numba.njit(parallel=True, fastmath=True)
def laplacian_3D_noflux(m_array, m_mapper):   
    laplacian = numpy.zeros(N_x*N_y*N_z);
    for itr in range(N_x*N_y*N_z): 
        discrete_laplacian = 0;
        i_ = m_mapper[itr][0]; j_ = m_mapper[itr][1];   k_ = m_mapper[itr][2];
        for i_inc in range (2):
            i_nflux = (i_+2*i_inc-1);
            j_nflux = (j_+2*i_inc-1);
            k_nflux = (k_+2*i_inc-1);
            if (i_nflux>-1 and i_nflux<N_x):
                discrete_laplacian+=m_array[k_*N_x*N_y+j_*N_x+i_nflux] - m_array[itr];                
            if (j_nflux>-1 and j_nflux<N_y):
                discrete_laplacian+=m_array[k_*N_x*N_y+j_nflux*N_x+i_] - m_array[itr];            
            if (k_nflux>-1 and k_nflux<N_z):
                discrete_laplacian+=m_array[k_nflux*N_x*N_y+j_*N_x+i_] - m_array[itr];
        laplacian[itr] = (1./delta**2)*discrete_laplacian;
    return laplacian;

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def divergence_anisotropic_3D_noflux(m_array, m_gamma,m_mapper):
    divergence = numpy.zeros(N_x*N_y*N_z);
    for itr in range(N_x*N_y*N_z):   
        i_ = m_mapper[itr][0]; j_ = m_mapper[itr][1];   k_ = m_mapper[itr][2];
        dx = (m_array[k_*N_x*N_y+j_*N_x+((i_+1) % N_x),0]-m_array[k_*N_x*N_y+j_*N_x+((i_-1) % N_x),0])/(2*delta);
        dy = (m_array[k_*N_x*N_y+((j_+1) % N_y)*N_x+i_,1]-m_array[k_*N_x*N_y+((j_-1) % N_y)*N_x+i_,1])/(2*delta);
        dz = (m_array[((k_+1) % N_z)*N_x*N_y+j_*N_x+i_,2]-m_array[((k_-1) % N_z)*N_x*N_y+j_*N_x+i_,2])/(2*delta);
        if (i_+1>N_x-1 or i_-1 <0):
            dx = 0;
        if (j_+1>N_y-1 or j_-1 <0):
            dy = 0;
        if (k_+1>N_z-1 or k_-1 <0):
            dz = 0;    
        divergence[itr] = dx + dy + m_gamma*dz;
    return divergence;

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def gradient_anisotropic_3D_noflux(m_array, m_gamma, m_mapper):
    gradient = numpy.zeros((N_x*N_y*N_z,3));
    for itr in range(N_x*N_y*N_z):
        i_ = m_mapper[itr][0]; j_ = m_mapper[itr][1];   k_ = m_mapper[itr][2];
        dx = (m_array[k_*N_x*N_y+j_*N_x+((i_+1) % N_x)]-m_array[k_*N_x*N_y+j_*N_x+((i_-1) % N_x)])/(2*delta);
        dy = (m_array[k_*N_x*N_y+((j_+1) % N_y)*N_x+i_]-m_array[k_*N_x*N_y+((j_-1) % N_y)*N_x+i_])/(2*delta);
        dz = m_gamma*(m_array[((k_+1) % N_z)*N_x*N_y+j_*N_x+i_]-m_array[((k_-1) % N_z)*N_x*N_y+j_*N_x+i_])/(2*delta); 
        if (i_+1>N_x-1 or i_-1 <0):
            dx = 0;
        if (j_+1>N_y-1 or j_-1 <0):
            dy = 0;
        if (k_+1>N_z-1 or k_-1 <0):
            dz = 0;            
        gradient[itr,0] = dx;
        gradient[itr,1] = dy;
        gradient[itr,2] = m_gamma*dz; 
    return gradient;

In [ ]:
def set_mapper():
    m_mapper = numpy.zeros((N_x*N_y*N_z,3));
    for i_ in range(N_x):        
        for j_ in range(N_y):                
            for k_ in range(N_z):
                m_mapper[k_*N_x*N_y+j_*N_x+i_][0] = i_;
                m_mapper[k_*N_x*N_y+j_*N_x+i_][1] = j_;                
                m_mapper[k_*N_x*N_y+j_*N_x+i_][2] = k_;
    return m_mapper.astype(int);

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def compute_M(m_grad_phi,m_grad_phi_Gamma):
    m_M = numpy.zeros((N_x*N_y*N_z,3));
    for itr in range(N_x*N_y*N_z):
        local_grad_phi = [m_grad_phi[itr,0],m_grad_phi[itr,1],m_grad_phi[itr,2]]; 
        local_G = compute_G(local_grad_phi,ep_xy,ep_z);     
        mod_del_phi = (local_grad_phi[0]**2+local_grad_phi[1]**2+local_grad_phi[2]**2);                            
        for q in range(3):
            m_M[itr,q]=mod_del_phi*local_G[q]+(compute_A_square(local_grad_phi,ep_xy,ep_z))*m_grad_phi_Gamma[itr,q];
    return m_M;

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def time_update_Phi(m_phi, m_update_phi, m_grad_phi, m_u, m_div_M, m_laplacian_phi):
    for itr in range(N_x*N_y*N_z):
        local_grad_phi = [m_grad_phi[itr,0],m_grad_phi[itr,1],m_grad_phi[itr,2]]; 
        surface_energy = m_div_M[itr];
        driving_force = lamd*compute_B(local_grad_phi,Gamma)*compute_g_prime(m_phi[itr])*m_u[itr];
        double_well_potential = -compute_f_prime(m_phi[itr]);
        curvature_scaling = 1/(compute_A_square(local_grad_phi,ep_xy,ep_z));
        m_update_phi[itr]= (double_well_potential+driving_force+surface_energy)*curvature_scaling+m_laplacian_phi[itr];                          
        m_phi[itr] += delta_t*m_update_phi[itr];
        m_phi[itr] = max(min(m_phi[itr],1),-1);
    return m_phi, m_update_phi;     

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def update_R(m_phi, m_R):
    for itr in range(N_x*N_y*N_z):   
        q = compute_q(phi[itr]);
        m_R[itr]*=q;
    return m_R;

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def update_u(m_grad_phi, m_update_phi, m_div_R, m_u, m_lapacian_u):
    for itr in range(N_x*N_y*N_z):
        local_grad_phi = [m_grad_phi[itr,0],m_grad_phi[itr,1],m_grad_phi[itr,2]]; 
        m_u[itr]+=delta_t*(m_div_R[itr]*D_tilde-0.5*L_sat*m_update_phi[itr]*compute_B(local_grad_phi,Gamma)+lapacian_u[itr]);
    return m_u;

In [ ]:
@numba.njit(parallel=True, fastmath=True)
def postprocess(m_phi, m_u, m_grad_phi):
    m_phi_postprocess = numpy.zeros((N_x,N_y,N_z));
    m_u_postprocess = numpy.zeros((N_x,N_y,N_z));
    for itr in range(N_x*N_y*N_z):
        m_phi_postprocess[mapper[itr][0],mapper[itr][1],mapper[itr][2]]=m_phi[itr]; 
        m_u_postprocess[mapper[itr][0],mapper[itr][1],mapper[itr][2]]=m_u[itr];  
    return m_phi_postprocess, m_u_postprocess;

In [ ]:
def write_vtk(time, m_phi_postprocess, m_u_postprocess):
    x = numpy.linspace(0, delta*(N_x-1), N_x);
    y = numpy.linspace(0, delta*(N_y-1), N_y);
    z = numpy.linspace(0, delta*(N_z-1), N_z); 
    gridToVTK("./results_folder/result_"+str(int(time/delta_t-1)), x, y, z, pointData = {"phi" : m_phi_postprocess,"u":m_u_postprocess});

In [ ]:
mapper = set_mapper();
u = numpy.zeros(N_x*N_y*N_z)+u_0;
phi = numpy.zeros(N_x*N_y*N_z)-1.;

# initialize phi and u fields
for itr in range(N_x*N_y*N_z): 
    i_ = mapper[itr][0]; j_ = mapper[itr][1]; k_ = mapper[itr][2]; 
    x_ = ((i_)**2+(j_)**2+0.1*(k_)**2)/(8)**2;
    phi[itr] = 1.-2.*math.tanh(x_**4);

# initialize with p random Fourier modes
p = 16;
random.seed(datetime.now());

# superimposed wave modes to satisfy periodicity
for p_ in range(p):
    freq = random.randint(1,int(p));
    weight = random.random();
    radians = freq*math.pi/N_x;
    for itr in range(N_x*N_y*N_z): 
        i_ = mapper[itr][0]; j_ = mapper[itr][1]; k_ = mapper[itr][2]; 
        u[itr] += u_0*(weight/(5*p))*((math.sin(i_*2*radians)+1)*(math.sin(j_*2*radians*3/2)+1)*(math.sin(k_*2*radians)+1)); 

update_phi = numpy.zeros(N_x*N_y*N_z);
lapacian_phi = numpy.zeros(N_x*N_y*N_z);
lapacian_u = numpy.zeros(N_x*N_y*N_z);

div_M = numpy.zeros(N_x*N_y*N_z);
div_R = numpy.zeros(N_x*N_y*N_z);

grad_phi_Gamma = numpy.zeros((N_x*N_y*N_z,3));
grad_phi = numpy.zeros((N_x*N_y*N_z,3)); 
grad_u = numpy.zeros((N_x*N_y*N_z,3)); 

M = numpy.zeros((N_x*N_y*N_z,3)); 
R = numpy.zeros((N_x*N_y*N_z,3)); 
local_M = numpy.zeros(3);
start = time.time()

In [ ]:
while (t<tmax):       
    grad_phi = gradient_anisotropic_3D_noflux(phi,1,mapper);
    grad_phi_Gamma = gradient_anisotropic_3D_noflux(phi,Gamma,mapper);  

    lapacian_phi = laplacian_3D_noflux(phi, mapper);

    lapacian_u = laplacian_3D_noflux(u, mapper);

    M = compute_M(grad_phi,grad_phi_Gamma);
    
    div_M = divergence_anisotropic_3D_noflux(M, Gamma,mapper);
     
    phi, update_phi = time_update_Phi(phi, update_phi, grad_phi, u, div_M, lapacian_phi);
 
    R = gradient_anisotropic_3D_noflux(u,Gamma,mapper); 

    R = update_R(phi, R);

    div_R = divergence_anisotropic_3D_noflux(R, Gamma,mapper);

    grad_phi = gradient_anisotropic_3D_noflux(phi,1,mapper);
    
    u = update_u(grad_phi, update_phi, div_R, u, lapacian_u); 

    print('update u at t =',t,' takes ', time.time() - start)
    print("time passed "+str(t))
    t+=delta_t;    

    if (int(t/delta_t-1)%sample_rate == 0):
        phi_postprocess, u_postprocess = postprocess(phi,u,grad_phi);
        write_vtk(t, phi_postprocess, u_postprocess);